## **FDS project, winter semester 2023**

#### Tommaso Leonardi, Arianna Paolini, Stefano Saravalle, Paolo Cursi, Pietro Signorino
<leonardi.1914546@studenti.uniroma1.it>, <paolini.1943164@studenti.uniroma1.it>, <saravalle.1948684@studenti.uniroma1.it>, <paoloc1999@gmail.com>, <signorino.2149741@studenti.uniroma1.it>

# **Student Performance Analysis & Prediction**

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn import linear_model

### **Data Preprocessing**

In [ ]:
#loading the dataset tables
courses = pd.read_csv('./data/courses.csv')  #22 rows for courses (modules) and their presentations
assess = pd.read_csv('./data/assessments.csv')  #206 rows of assessments for module-presentations (including the final exam)
results = pd.read_csv('./data/studentAssessment.csv') #173,912 rows for the scores obtained by students in the asssesments
studs = pd.read_csv('./data/studentInfo.csv') #32,593 rows for demographic information on students and their results in module-presentations
registr = pd.read_csv('./data/studentRegistration.csv') #32,593 rows for student registration/unregistration on module-presentations
vle = pd.read_csv('./data/studentVle.csv') #10,655,280 rows for daily student interactions with online resources for a module-presentation
materials = pd.read_csv('./data/vle.csv') #6,364 rows for the materials available on the Virtual Learning Environment

The _Open University Learning Analytics dataset_ that we are considering has the following structure: 

 <img src="https://analyse.kmi.open.ac.uk/resources/images/model.png" alt="dataset structure" style="height: 500px; width:500px;"/>


(https://analyse.kmi.open.ac.uk/open_dataset)



In [ ]:
#DONE: definire i task -> regressione/classificazione dello score per ogni assessment
#DONE: trasformare valori categorici in numeri
#DONE: normalizzare i dati
#?: pulire i dati
#DONE: mostrare la distribuzione dei dati con grafici
#DONE: fare split tra training e test set (considerare cross validation)


Since our goal is to predict the score of each student in any assessment belonging to a specific module presentation, we consider the demographic information about students (from the table *studentInfo*) and their accessess to online resources in the Virtual Learning Environment (VLE) for each course (from the tables *studentVle* and *vle*) as features for our models. 

We also take in account the assessment type and weigth (from the table *assessments*) and the time the student spent before submitting it (from the table *studentAssessment*). 

The target value to predict is the score from the *studentAssessment* table, which ranges from 0 to 100.

In [ ]:
#dropping some features
studs = studs.drop("final_result", axis=1)
registr = registr.drop("date_unregistration", axis=1)
materials = materials.drop(["week_from", "week_to"], axis=1)
vle = vle.drop("date", axis=1)
results = results.drop("is_banked", axis=1)

#remove Nan values in studs 'imd_band' (amounts to around 7000 total assessment results)
studs=studs.dropna()

#remove students with 'date registration' that is null (amounts to 7 total assessment results)
registr=registr.dropna()

#match info about a student and his date of registration to a module presentation
studs = studs.merge(registr, how="inner", on=["code_module", "code_presentation","id_student"])

#match a student's interactions with an online resource with the type of the resource
vle = vle.merge(materials, how="inner", on=["code_module", "code_presentation","id_site"] )
vle = vle.drop("id_site", axis=1)

#group the interactions by resource type and add a feature to consider the total sum of clicks for each resource type
temp = vle.groupby(["code_module", "code_presentation", "id_student", "activity_type"]).sum().reset_index()
for x in temp["activity_type"].unique():
    temp[x+"_clicks"] = np.where(temp["activity_type"]==x, temp["sum_click"], 0)
temp = temp.drop(["activity_type","sum_click"], axis=1)
temp = temp.groupby(["code_module", "code_presentation", "id_student"]).sum().reset_index()

#match student's information with his interactions on the VLE for a specific module presentation and fill nan with 0
studs = studs.merge(temp, how="left", on=["code_module", "code_presentation", "id_student"])
studs= studs.fillna(0)

#remove assessments without 'date'=nan and results with 'score'=nan
assess=assess.dropna()
results=results.dropna()
#a=results.merge(assess[assess['date'].isnull()], how="inner", on=["id_assessment"]) #only 2865 instances are removed

#match assessments with students scores
assess = assess.merge(results, how="inner", on="id_assessment")

#substitute date in assessment and date_submitted in results with their difference (to be considered as a time delay from expected submission)
assess["submission_delay"] = assess["date_submitted"] - assess["date"] #"date" is the deadline for the assessment
assess = assess.drop(["date","date_submitted"], axis=1)

#match students with their results
df = studs.merge(assess, how="inner", on=["code_module", "code_presentation","id_student"])

#remove ids from the features
df = df.drop(["code_module","code_presentation","id_student","id_assessment"], axis=1) 
print(df.shape) #163387 total rows

Some of the features in the dataset have *string* values (e.g. 'gender' has values {'M', 'F'}). 

We convert those features to integer values. The list *to_be_converted* contains the names of such features.

In [ ]:
df["highest_education"] = df["highest_education"].replace({"No Formal quals":0, "Lower Than A Level":1, "A Level or Equivalent": 2, 
                                                           "HE Qualification":3, "Post Graduate Qualification":4 })

df["imd_band"] = df["imd_band"].replace({"0-10%":0, "10-20":1, "20-30%": 2, "30-40%":3, "40-50%":4, "50-60%":5,
                                          "60-70%":6, "70-80%":7, "80-90%":8, "90-100%":9 })

df["age_band"] = df["age_band"].replace({"0-35":0, "35-55":1, "55<=": 2})

to_be_converted = ["gender", "region", "disability", "assessment_type"]

for column_name in to_be_converted:

    values = set(df[column_name].tolist())
    print(f"Values in '{column_name}' column: {values}")

    mapping = {x : y for y,x in enumerate(values)}
    print(f"Mapping from string values to numerical using the following dictionary: {mapping}")

    df[column_name] = df[column_name].map(mapping)

    print("\n==================================================\n")


In [ ]:
#showing the feature values
df.hist(layout=(7,5),figsize=(18, 18), bins=25)
plt.show()

With the previous step we have obtained a dataset containing only *integer values*.

We normalize every feature $f$ with values $vals$ and
- mean:  $mn$
- standard deviation:  $std$

with the following algorithm.

For each $v \in vals$:
$$
v := \frac{v - mn}{std}
$$

<!-- This step ensures that every value in the dataset is in the range $[0,1]$. -->

Let's also save the score for later computations using Neural Networks

In [ ]:
y_nn = df["score"].tolist()

In [ ]:
for col in df:
    print(f"Normalizing column '{col}'")
    print("\n==================================================\n")

    df[col] = (df[col] - df[col].mean()) / df[col].std()

In [ ]:
#showing the feature values after normalization
df.hist(layout=(7,5),figsize=(18, 18), bins=25)
plt.show()

for col in df:
    print(f"Column '{col}' has max value of {df[col].max()}")


In [ ]:
df

To better understand the influence of each feature on the value we want to predict, we use some scatterplots to show the relationship between each single feature of the dataset and the score value for an assessment.

In [ ]:
scatPlotRows=11
scatPlotColumns=3
figure, axis = plt.subplots(scatPlotRows,scatPlotColumns,figsize=(scatPlotColumns*4,scatPlotRows*4)) 
for i,col in enumerate(df):  
    if col=="score": continue  
    axis[i//scatPlotColumns,i%scatPlotColumns].scatter(df[col], df["score"],s=2)
    axis[i//scatPlotColumns,i%scatPlotColumns].set_xlabel(col)
    axis[i//scatPlotColumns,i%scatPlotColumns].set_ylabel("score")
plt.show()

We can see that there isn't a linear relationship between features and target, as we could imagine, so we don't expect linear models to perform very well.

We get the training and test sets for our ML models by splitting the dataset according to a given ratio between training and test samples. We choose to use 80% of the original number of samples for training and the remaining 20% for testing.

In [ ]:
#using scikit-learn to split the dataset into train and test sets
X = df.drop("score", axis=1)
y = df["score"].copy()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, test_size=0.2, random_state=7) #130709 train samples, 32678 test


X_train.to_csv("sets/X_train.csv", index=False)
X_test.to_csv("sets/X_test.csv", index=False)
y_train.to_csv("sets/y_train.csv", index=False)
y_test.to_csv("sets/y_test.csv", index=False)

## Regression

### Linear regression


In [ ]:
linearReg = linear_model.LinearRegression()
linearReg.fit(X_train, y_train)
y_pred = linearReg.predict(X_test)

print("RMSE: %.2f" % metrics.mean_squared_error(y_test, y_pred))
print("R2: %.2f" % metrics.r2_score(y_test, y_pred))

### Decision Trees

Decision trees are a simple ML model that we can use for the regression task of predicting the score of some student in an assessment.
This model should be able to deal with the non-linear relationship between the features and the target, thus achieving better results compared to linear regression. However, it might be prone to overfitting.

In [ ]:
#using the scikit-learn implementation of the model
from sklearn.tree import DecisionTreeRegressor

min_samples_leaf = 15 #minimum number of samples required in a leaf -> may smooth the model
min_samples_split = 10 #minimum number of samples to split an internal node

decTree = DecisionTreeRegressor(min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split)

decTree.fit(X_train, y_train)

We evaluate the model by considering the RMSE error and the R2 score.

In [ ]:
y_pred_trees = decTree.predict(X_test)
y_pred_train_trees=decTree.predict(X_train)
#de-normlize oreds and test
y_pred_trees=(y_pred_trees*np.std(y_nn))-np.mean(y_nn)
y_pred_train_trees=(y_pred_train_trees*np.std(y_nn))-np.mean(y_nn)
y_test_deNorm=(y_test*np.std(y_nn))-np.mean(y_nn)
y_train_deNorm=(y_train*np.std(y_nn))-np.mean(y_nn)

RMSE_trees = metrics.mean_squared_error(y_test_deNorm, y_pred_trees, squared=False)
R2_trees = metrics.r2_score(y_test_deNorm, y_pred_trees)

RMSE_trees_train = metrics.mean_squared_error(y_train_deNorm, y_pred_train_trees, squared=False)
R2_trees_train = metrics.r2_score(y_train_deNorm, y_pred_train_trees)

In [ ]:
print("RMSE: "+str(RMSE_trees))
print("R2: "+str(R2_trees))

print("RMSE for training: "+str(RMSE_trees_train))
print("R2 for training: "+str(R2_trees_train))

## Multinomial classification

### Neural Networks

Feature engineering using the following [link](https://www.kaggle.com/code/devassaxd/student-performance-prediction-complete-analysis)

In [ ]:
studInfo=pd.read_csv("data/studentInfo.csv")
assessments=pd.read_csv("data/assessments.csv")
studAss=pd.read_csv("data/studentAssessment.csv")
studVle=pd.read_csv("data/studentVle.csv")
vle=pd.read_csv("data/vle.csv")

#### Assessments
The performance in each assessement is a good indicator of the students' knowledge of the course and, as it composes the grade for the final evaluation, it's interesting to make it a feature in the final model. But, as there are many different courses, each with a different structure, it's unfeasible to create a feature for each assessment. In order to include the assessments, we will build 2 features: One of them is the final grade given by the score and the weight of each assessment. The other is a pass rate, created on the premise that a student must get at least 40% score on an assessment to pass it, calculating the percentage of assessments the student sucessfully passed. We also will split final exams from the other assessments, given their status and participation in the final evaluation is different from the other assessments.

In [ ]:
exams=assessments[assessments["assessment_type"]=="Exam"]
others=assessments[assessments["assessment_type"]!="Exam"]
amounts=others.groupby(["code_module","code_presentation"]).count()["id_assessment"] 
amounts=amounts.reset_index()
amounts.head()

In [ ]:
#Function to determine whether a student passed a given assessment
def pass_fail(grade):
    if grade>=40:
        return True
    else:
        return False
#Creating the stud_ass dataframe to join infos about the assessment weights and their respective grades
stud_ass=pd.merge(studAss,others,how="inner",on=["id_assessment"])
stud_ass["pass"]=stud_ass["score"].apply(pass_fail)
stud_ass["weighted_grade"]=stud_ass["score"]*stud_ass["weight"]/100

In [ ]:
#Final assessment average per student per module
avg_grade=stud_ass.groupby(["id_student","code_module","code_presentation"]).sum()["weighted_grade"].reset_index()
avg_grade.head()

In [ ]:
#Pass rate per student per module
pass_rate=pd.merge((stud_ass[stud_ass["pass"]==True].groupby(["id_student","code_module","code_presentation"]).count()["pass"]).reset_index(),amounts,how="left",on=["code_module","code_presentation"])
pass_rate["pass_rate"]=pass_rate["pass"]/pass_rate["id_assessment"]
pass_rate.drop(["pass","id_assessment"], axis=1,inplace=True)
pass_rate.head()

In [ ]:
#Final exam scores
stud_exams=pd.merge(studAss,exams,how="inner",on=["id_assessment"])
stud_exams["exam_score"]=stud_exams["score"]
stud_exams.drop(["id_assessment","date_submitted","is_banked", "score","assessment_type","date","weight"],axis=1,inplace=True)
stud_exams.head()

#### VLE
The datasets referring to the VLE (Virtual Learning Environment) contain the interaction feed of the students with the content available for reference throughout the duration of the period. From this data we can infer how in touch a student was with their subjects, whether they studied it on a solid basis and how they used the content.

In [ ]:
vle

In [ ]:
vle[~vle["week_from"].isna()]
#Only 1121 from the 6364 entries have the reference week for the materials (the week in which they would be used in course.)
#With this in mind, the construction of a metric to track study commitment becomes impractical

In [ ]:
studVle.head()

In [ ]:
#Here we can track the average time after the start of the course the student took to use the materials
#and the average amount of clicks per material
avg_per_site=studVle.groupby(["id_student","id_site","code_module","code_presentation"]).mean().reset_index()
avg_per_site.head()

In [ ]:
#General average per student per module
avg_per_student=avg_per_site.groupby(["id_student","code_module","code_presentation"]).mean()[["date","sum_click"]].reset_index()
avg_per_student.head()

#### StudentInfo

The studentInfo table contains various info about the students, but the relevant ones for this analysis are:

- The amount of times the student has already tried to finish the module
- The students' final result

The last one is our interest variable as we build our prediction model

In [ ]:
#Removing the cases where the student has withdrawn their registration to the module
studInfo=studInfo[studInfo["final_result"]!="Withdrawn"]
studInfo=studInfo[["code_module","code_presentation","id_student","num_of_prev_attempts","final_result"]]
studInfo.head()

#### Compiling all relevant tables

In [ ]:
df_1=pd.merge(avg_grade,pass_rate,how="inner",on=["id_student","code_module","code_presentation"])
assessment_info=pd.merge(df_1, stud_exams, how="inner", on=["id_student","code_module","code_presentation"])
assessment_info.head()

In [ ]:
df_2=pd.merge(studInfo,assessment_info,how="inner",on=["id_student","code_module","code_presentation"])
final_df=pd.merge(df_2,avg_per_student,how="inner", on=["id_student","code_module","code_presentation"])
final_df.drop(["id_student","code_module","code_presentation"],axis=1,inplace=True)
final_df.head()
#The final dataframe only has information relevant to the problem

Delete some outliers to have the most uniform dataset possible

In [ ]:
final_df=final_df[final_df["sum_click"]<=10]
final_df=final_df[final_df["num_of_prev_attempts"]<=4]
final_df.head()

In [ ]:
X=final_df.drop("final_result", axis=1)
y=final_df["final_result"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class StudentDataset(Dataset):

    def __init__(self, X, y, transform=None, percentage = 1):
        self.X = X.values.tolist()
        self.y = y

        if percentage != 1:
            self.X = self.X[:int(len(self.X)*percentage)]
            self.y = self.y[:int(len(self.y)*percentage)]

    def __len__(self):
        return len(self.X)
        

    def __getitem__(self, idx):
        x_sample = torch.tensor(self.X[idx])
        y_sample = torch.zeros(10)
        if int(self.y[idx]/10) == 10:
            y_sample[9] = 1
        else:
            y_sample[int(self.y[idx]/10)] = 1

        return x_sample, y_sample
        

In [ ]:
dataset_train = StudentDataset(X_train, y_train)
dataset_test = StudentDataset(X_test, y_test)

In [ ]:
dataloader_train = DataLoader(dataset_train, batch_size=16,
                        shuffle=True, num_workers=0)

dataloader_test = DataLoader(dataset_test, batch_size=16,
                        shuffle=True, num_workers=0)

In [ ]:
x, y = dataset_train[0]
in_features = len(x)
out_features = len(y)

In [ ]:
class StudentScoreClassifier(nn.Module):
    def __init__(self):
        super(StudentScoreClassifier, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(in_features, in_features * 4),
            nn.ReLU(),
            nn.Linear(in_features*4, in_features * 8),
            nn.ReLU(),
            nn.Linear(in_features*8, in_features * 4),
            nn.ReLU(),
            nn.Linear(in_features*4, out_features),
            nn.Softmax()
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = StudentScoreClassifier().to(device)
epochs = 1000

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    corrects = 0.0
    for i, data in enumerate(dataloader_train):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        _, preds = torch.max(outputs.data, 1)
        _, correct_labels = torch.max(labels.data ,1)
        """print(preds)
        print(correct_labels)
        print("=================================")"""

        corrects += float(torch.sum(preds == correct_labels))
        
    print("EPOCH {} \t LOSS: {:.3f} \t ACCURACY: {:.3f}".format(epoch+1, running_loss/len(dataset_train), corrects/len(dataset_train)) )

print('Finished Training')

In [ ]:
model.eval()

for i, data in enumerate(dataloader_test):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)

    # forward + backward + optimize
    outputs = model(inputs)

    _, preds = torch.max(outputs.data, 1)
    _, correct_labels = torch.max(labels.data ,1)

    corrects += float(torch.sum(preds == correct_labels))

print("Precision: {:.3f}".format(corrects/len(dataset_test)))